## Load CSV into Table

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [ ]:
%sh 
echo 'Files in  Directory:'
echo
ls /enter_path_here/


Files in SBPI Directory:

CATEGORY.csv
CATEGORY_CH.csv
CATEGORY_US.csv
CURRENCY_MARKET.csv
CURRENCY_MARKET_CH.csv
CURRENCY_MARKET_US.csv

In [ ]:
file_path =  dbutils.fs.ls('ENTER FILE PATH')
file_names = []
file_paths = []
for f in file_path:
  #print('file_name: ',f.name,' file_path: ', f.path[5:],' file_size (bytes): ',f.size)
  file_names.append(f.name)
  file_paths.append(f.path[5:])
 
print('file names: ',file_names)
print('file paths: ',file_paths)


file names: ['CATEGORY.csv', 'CATEGORY.html', 'CATEGORY_CH.csv', 'CATEGORY_US.csv', 'CURRENCY_MARKET.csv', 'CURRENCY_MARKET_CH.csv', 'CURRENCY_MARKET_US.csv', 'SC_STD_COUPON.html', 'SC_STD_NEW_ISSUE.html']
file paths: ['/SBPI/in_files/CATEGORY.csv', '/SBPI/in_files/CATEGORY.html', '/SBPI/in_files/CATEGORY_CH.csv', '/SBPI/in_files/CATEGORY_US.csv', '/SBPI/in_files/CURRENCY_MARKET.csv', '/SBPI/in_files/CURRENCY_MARKET_CH.csv', '/SBPI/in_files/CURRENCY_MARKET_US.csv', '/SBPI/in_files/SC_STD_COUPON.html', '/SBPI/in_files/SC_STD_NEW_ISSUE.html']

In [ ]:

# File location and type
files_location = "FILE PATH"
file_type = "csv"
print(files_location)

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = []
# The applied options are for CSV files. For other file types, these will be ignored.
i=-1
for f in file_paths[:]:
  i=1+i
  df[i] = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(f)\
  

/SBPI/in_files/

In [ ]:
%sql
use database sbpi

In [ ]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "sbpi.CATEGORY_csv"

df.write.format("delta").saveAsTable(permanent_table_name)

files = []
for i in file_names:
  schema='sbpi.'
  files.append(schema+i)

print(files)

files = [elem.replace('.csv', '') for elem in files]

i=0
for table_name in files:
  t = str(table_name)
  df[i].write.format("delta").saveAsTable(t)
  i+=1

